## Stock Data Analysis and Model Building
#### Script for creating and identifying features/crash indicators and building a regression based model for crash prediction

#### **Step - 1: Importing and installing necessary libraries**






In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta


#### **Step - 2: Using yfinance (Yahoo Finance) to extract historical stock data for the last 5 years**

In [2]:
tickers_yf = {"META" : "Meta",
              "AAPL" : "Apple",
              "AMZN" : "Amazon",
              "NFLX" : "Netflix",
              "GOOG" : "Google",
              "MSFT" : "Microsoft",
              "TSLA" : "Tesla",
              "RELIANCE.NS" : "Reliance",
              "INFY.NS" : "Infosys",
              "HDFCBANK.NS" : "HDFC"}


def get_historical_data():
  df_yf = pd.DataFrame()
  for ticker, name in tickers_yf.items():
    print(f"Fetching historical data for {name} ({ticker})")
    stock = yf.Ticker(ticker)
    start_date = date.today() - relativedelta(years = 5)
    end_date = date.today()
    hist = stock.history(start = start_date, end = end_date)      #end is exclusive of the end-date
    hist["Company"] = name
    hist["Date"] = hist.index.date
    df_yf = pd.concat([df_yf, hist])
  return df_yf.reset_index(drop = True)

df1 = get_historical_data()

Fetching historical data for Meta (META)
Fetching historical data for Apple (AAPL)
Fetching historical data for Amazon (AMZN)
Fetching historical data for Netflix (NFLX)
Fetching historical data for Google (GOOG)
Fetching historical data for Microsoft (MSFT)
Fetching historical data for Tesla (TSLA)
Fetching historical data for Reliance (RELIANCE.NS)
Fetching historical data for Infosys (INFY.NS)
Fetching historical data for HDFC (HDFCBANK.NS)


In [3]:
print(df1.shape)
df1.head()

(12502, 9)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company,Date
0,240.232084,243.484247,236.751181,238.969009,19133600,0.0,0.0,Meta,2020-07-15
1,236.701442,240.440921,235.040555,239.615448,18880700,0.0,0.0,Meta,2020-07-16
2,239.685078,242.827841,236.562211,240.709457,18152400,0.0,0.0,Meta,2020-07-17
3,238.750192,244.707515,234.354310,244.080948,20872200,0.0,0.0,Meta,2020-07-20
4,244.876581,245.562819,238.770083,240.430969,19364200,0.0,0.0,Meta,2020-07-21


In [4]:
df2 = df1[["Date", "Company", "Close", "Volume"]]
df2.head()

,Date,Company,Close,Volume
0,2020-07-15,Meta,238.969009,19133600
1,2020-07-16,Meta,239.615448,18880700
2,2020-07-17,Meta,240.709457,18152400
3,2020-07-20,Meta,244.080948,20872200
4,2020-07-21,Meta,240.430969,19364200


In [5]:
df2["Date"] = pd.to_datetime(df2["Date"], format = "%Y-%m-%d", errors = "coerce")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12502 entries, 0 to 12501
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     12502 non-null  datetime64[ns]
 1   Company  12502 non-null  object        
 2   Close    12502 non-null  float64       
 3   Volume   12502 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 390.8+ KB


/tmp/ipython-input-5-2800292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Date"] = pd.to_datetime(df2["Date"], format = "%Y-%m-%d", errors = "coerce")


### **Step - 3: Calculating the Crash Indicators / Features from the Historical Stock Data**

In [6]:
def compute_rsi(series, window=14):
  delta = series.diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window = window).mean()
  avg_loss = loss.rolling(window = window).mean()
  rs = avg_gain/avg_loss

  return 100 - (100/(1+rs))


def max_drawdown(x):
    roll_max = x.cummax()
    drawdowns = (x - roll_max) / roll_max
    return drawdowns.min()


# To prevent Division by Zero Errors
def safe_pct_change(series, periods):
    shifted = series.shift(periods)
    shifted = shifted.replace(0, np.nan)
    return (series - shifted) / shifted


def extract_crash_features(df):
  df = df.copy()

  # Sorting the dataframe by date and company for further calculations
  df.sort_values(by = ["Company", "Date"], inplace=True)

  # Calculating 1-day, 5-day and 20-day Returns
  df["Return_1d"] = df.groupby("Company")["Close"].pct_change()
  df["Return_5d"] = df.groupby("Company")["Close"].pct_change(periods=5)
  df["Return_20d"] = df.groupby("Company")["Close"].pct_change(periods = 20)

  # Volatility over 10-day and 20-day periods
  df["Volatility_10"] = df.groupby("Company")["Close"].transform(lambda x: x.rolling(window=10).std())
  df["Volatility_20"] = df.groupby("Company")["Close"].transform(lambda x: x.rolling(window = 20).std())

  # Moving Average 20 and Moving Average 50
  df["MA_20"] = df.groupby("Company")["Close"].transform(lambda x: x.rolling(window = 20).mean())
  df["MA_50"] = df.groupby("Company")["Close"].transform(lambda x: x.rolling(window = 50).mean())
  df["MA_Crossover"] = df["MA_20"] - df["MA_50"]

  # MACD (Moving Average Convergence Divergence) and Signal
  ema_12 = df.groupby("Company")["Close"].transform(lambda x: x.ewm(span = 12, adjust = False).mean())
  ema_26 = df.groupby("Company")["Close"].transform(lambda x: x.ewm(span = 26, adjust = False).mean())
  df["MACD"] = ema_12 - ema_26
  df["MACD_Signal"] = df.groupby("Company")["MACD"].transform(lambda x: x.ewm(span = 9, adjust = False).mean())

  # RSI over 14-day windows
  df["RSI_14"] = df.groupby("Company")["Close"].transform(compute_rsi)

  # Crash Flag - Target Variable; short-term crash indicator (for >5% drop)
  df["Crash_Flag"] = df.groupby("Company")["Close"].pct_change().transform(lambda x: (x < -0.05).astype(int)).fillna(0)

  # Volume Surge and Volume Change
  df["Volume_Rolling_Mean"] = df.groupby("Company")["Volume"].transform(lambda x: x.rolling(window = 10).mean())
  df["Volume_Surge"] = df["Volume"]/df["Volume_Rolling_Mean"]
  df["Volume_Change_3d"] = df.groupby("Company")["Volume"].transform(lambda x: safe_pct_change(x, periods = 3))

  # Drawdown % and Max Drawdown within a 30-day period
  df["Drawdown_Pct"] = df.groupby("Company")["Close"].transform(lambda x: (x - x.cummax())/x.cummax())
  df["Max_Drawdown_30d"] = df.groupby("Company")["Close"].transform(lambda x: x.rolling(window=30).apply(max_drawdown))

  # Dropping NaNs and infs from rolling calculations
  df.replace([np.inf, -np.inf], np.nan, inplace = True)
  df.dropna(subset = ["Return_1d", "Volatility_10", "Volatility_20", "MACD", "MACD_Signal", "RSI_14", "Crash_Flag",
                      "Volume_Rolling_Mean","Volume_Surge", "Volume_Change_3d", "Drawdown_Pct", "Max_Drawdown_30d",
                      "MA_20", "MA_50", "MA_Crossover"], inplace=True)

  return df.reset_index(drop=True)

In [7]:
df3 = extract_crash_features(df2)
print(df3.shape)
df3.head()


(12009, 21)


,Date,Company,Close,Volume,Return_1d,Return_5d,Return_20d,Volatility_10,Volatility_20,MA_20,...,MA_Crossover,MACD,MACD_Signal,RSI_14,Crash_Flag,Volume_Rolling_Mean,Volume_Surge,Volume_Change_3d,Drawdown_Pct,Max_Drawdown_30d
0,2020-09-23,Amazon,149.992996,113054000,-0.041269,-0.025418,-0.103580,4.052336,9.368180,161.219348,...,2.485000,-3.018738,-1.468553,28.277796,0,115098200.0,0.982240,-0.364337,-0.150530,-0.163259
1,2020-09-24,Amazon,150.989502,110588000,0.006644,0.003676,-0.122626,3.636339,9.266949,160.164198,...,1.418930,-3.113813,-1.797605,33.880513,0,115495600.0,0.957508,-0.096193,-0.144887,-0.163259
2,2020-09-25,Amazon,154.756500,92304000,0.024949,0.047453,-0.089668,3.548090,9.039482,159.402023,...,0.561525,-2.852315,-2.008547,40.781846,0,114538000.0,0.805881,-0.335828,-0.123553,-0.163259
3,2020-09-28,Amazon,158.702499,84484000,0.025498,0.072144,-0.066950,3.990451,8.682424,158.832648,...,-0.219930,-2.300152,-2.066868,51.191209,0,113927200.0,0.741561,-0.252711,-0.101205,-0.163259
4,2020-09-29,Amazon,157.244003,69916000,-0.009190,0.005078,-0.088694,3.916584,8.062282,158.067448,...,-0.933170,-1.957681,-2.045031,43.323376,0,112875800.0,0.619406,-0.367780,-0.109465,-0.163259


In [8]:
print(df3.shape)
print(df3.info())
df3.head()

(12009, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 12009 non-null  datetime64[ns]
 1   Company              12009 non-null  object        
 2   Close                12009 non-null  float64       
 3   Volume               12009 non-null  int64         
 4   Return_1d            12009 non-null  float64       
 5   Return_5d            12009 non-null  float64       
 6   Return_20d           12009 non-null  float64       
 7   Volatility_10        12009 non-null  float64       
 8   Volatility_20        12009 non-null  float64       
 9   MA_20                12009 non-null  float64       
 10  MA_50                12009 non-null  float64       
 11  MA_Crossover         12009 non-null  float64       
 12  MACD                 12009 non-null  float64       
 13  MACD_Signal        

,Date,Company,Close,Volume,Return_1d,Return_5d,Return_20d,Volatility_10,Volatility_20,MA_20,...,MA_Crossover,MACD,MACD_Signal,RSI_14,Crash_Flag,Volume_Rolling_Mean,Volume_Surge,Volume_Change_3d,Drawdown_Pct,Max_Drawdown_30d
0,2020-09-23,Amazon,149.992996,113054000,-0.041269,-0.025418,-0.103580,4.052336,9.368180,161.219348,...,2.485000,-3.018738,-1.468553,28.277796,0,115098200.0,0.982240,-0.364337,-0.150530,-0.163259
1,2020-09-24,Amazon,150.989502,110588000,0.006644,0.003676,-0.122626,3.636339,9.266949,160.164198,...,1.418930,-3.113813,-1.797605,33.880513,0,115495600.0,0.957508,-0.096193,-0.144887,-0.163259
2,2020-09-25,Amazon,154.756500,92304000,0.024949,0.047453,-0.089668,3.548090,9.039482,159.402023,...,0.561525,-2.852315,-2.008547,40.781846,0,114538000.0,0.805881,-0.335828,-0.123553,-0.163259
3,2020-09-28,Amazon,158.702499,84484000,0.025498,0.072144,-0.066950,3.990451,8.682424,158.832648,...,-0.219930,-2.300152,-2.066868,51.191209,0,113927200.0,0.741561,-0.252711,-0.101205,-0.163259
4,2020-09-29,Amazon,157.244003,69916000,-0.009190,0.005078,-0.088694,3.916584,8.062282,158.067448,...,-0.933170,-1.957681,-2.045031,43.323376,0,112875800.0,0.619406,-0.367780,-0.109465,-0.163259


In [9]:
df3.corr(numeric_only=True)[["Close", "Crash_Flag"]].sort_values(by = ["Crash_Flag", "Close"], ascending = False)

,Close,Crash_Flag
Crash_Flag,-0.073874,1.000000
Volume,-0.485057,0.215575
Volume_Surge,-0.000340,0.196050
Volume_Change_3d,0.056964,0.152845
Volume_Rolling_Mean,-0.522176,0.138069
Volatility_10,0.694168,0.026585
Volatility_20,0.733048,0.010236
MA_Crossover,0.178880,-0.053431
MACD_Signal,0.192262,-0.057325
MA_50,0.995736,-0.064663


### **Step - 4: Model Experimentation - choosing the right model**

In [10]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import warnings

warnings.filterwarnings("ignore")

In [11]:
def train_evaluate_classifiers(df, feature_cols, target_col = "Crash_Flag", n_splits = 5):
  X = df[feature_cols]
  y = df[target_col]

  # Defining Classifiers
  models = {"Logistic Regression": LogisticRegression(class_weight = "balanced"),
            "Random Forest": RandomForestClassifier(class_weight= "balanced", n_estimators = 100),
            "XGBoost": XGBClassifier(use_label_encoder = False, eval_metric = "logloss"),
            "LightGBM": LGBMClassifier()}

  results_summary = {
    "Model": [],
    "Fold": [],
    "Accuracy": [],
    "ROC_AUC": [],
    "Precision_1": [],
    "Recall_1": [],
    "F1_1": [],
    "Support_1": [] }


  conf_matrices = {"Model": [], "Fold": [], "Matrix": []}

  tscv = TimeSeriesSplit(n_splits= n_splits)
  fold = 1
  for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Scaling for Logistic Regression model
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Training and Evaluation
    for name, model in models.items():

      if name == "Logistic Regression":
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_crash_prob = model.predict_proba(X_test_scaled)[:,1]
      else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_crash_prob = model.predict_proba(X_test)[:, 1]

      # Classification report
      report_dict = classification_report(y_test, y_pred, output_dict= True)
      report_df = pd.DataFrame(report_dict).transpose().round(4)

      # Confusion matrix
      #print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
      cm = confusion_matrix(y_test, y_pred)
      conf_matrices["Model"].append(name)
      conf_matrices["Fold"].append(fold)
      conf_matrices["Matrix"].append(cm)


      results_summary["Model"].append(name)
      results_summary["Fold"].append(fold)
      results_summary["Accuracy"].append(report_df.loc["accuracy", "f1-score"])
      results_summary["ROC_AUC"].append(round(roc_auc_score(y_test, y_crash_prob), 4))
      results_summary["Precision_1"].append(report_df.loc["1", "precision"])
      results_summary["Recall_1"].append(report_df.loc["1", "recall"])
      results_summary["F1_1"].append(report_df.loc["1", "f1-score"])
      results_summary["Support_1"].append(report_df.loc["1", "support"])

    fold += 1

  print("\n Average Confusion Matrices Across Folds: \n")
  for model_name in set(conf_matrices["Model"]):
    matrices = [cm for m, cm in zip(conf_matrices["Model"], conf_matrices["Matrix"]) if m == model_name]
    avg_cm = np.mean(matrices, axis=0).round().astype(int)
    print(f"\n{model_name}:\n{avg_cm}")

  summary_df = pd.DataFrame(results_summary)
  print("\n Final Summary Across Folds (Mean):\n")
  print(summary_df.groupby("Model").mean().round(4).sort_values("ROC_AUC", ascending=False))
  print("\n Final Summary Across Folds (Standard Deviation):\n")
  print(summary_df.groupby("Model").std().round(4))


In [12]:
df3.columns

Index(['Date', 'Company', 'Close', 'Volume', 'Return_1d', 'Return_5d',
       'Return_20d', 'Volatility_10', 'Volatility_20', 'MA_20', 'MA_50',
       'MA_Crossover', 'MACD', 'MACD_Signal', 'RSI_14', 'Crash_Flag',
       'Volume_Rolling_Mean', 'Volume_Surge', 'Volume_Change_3d',
       'Drawdown_Pct', 'Max_Drawdown_30d'],
      dtype='object')

In [13]:
feature_cols = ["Return_1d", "Volatility_10", "Volatility_20", "MACD", "MACD_Signal", "RSI_14", "Volume_Rolling_Mean",
                "Volume_Surge", "Volume_Change_3d", "Drawdown_Pct", "Max_Drawdown_30d",
                "MA_20", "MA_50", "MA_Crossover"]

train_evaluate_classifiers(df3, feature_cols)

[LightGBM] [Info] Number of positive: 28, number of negative: 1976
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3566
[LightGBM] [Info] Number of data points in the train set: 2004, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.013972 -> initscore=-4.256625
[LightGBM] [Info] Start training from score -4.256625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

#### **Step - 5: Training the Best Model - XGBoost**

In [14]:
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV

In [15]:
X = df3[feature_cols]
y = df3["Crash_Flag"]

tscv = TimeSeriesSplit(n_splits = 5)
train_index, _ = list(tscv.split(X))[0]
y_train = y.iloc[train_index]

# Calculating scale_pos_weight based on training data
pos = sum(y_train == 1)
neg = sum(y_train == 0)
scale_pos_weight = neg / pos

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)

param_grid = {'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0] }


search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    scoring='f1',
    n_iter=10,
    cv=tscv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X, y)

print("Best hyperparameters:", search.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.8}


In [16]:
best_params = search.best_params_

final_model = XGBClassifier(use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    **best_params )

final_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### **Step - 6: Save the Model as a pickle file**

In [17]:
import joblib

joblib.dump(final_model, "xgb_stock_crash_predictor.pkl")

['xgb_stock_crash_predictor.pkl']

In [18]:
from google.colab import files

files.download("xgb_stock_crash_predictor.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### **Step - 7: Generating Crash Predictions**

In [19]:
df3["Crash_Prediction"] = final_model.predict(X)
df3["Crash_Probability"] = final_model.predict_proba(X)[:, 1]
df3.head()

,Date,Company,Close,Volume,Return_1d,Return_5d,Return_20d,Volatility_10,Volatility_20,MA_20,...,MACD_Signal,RSI_14,Crash_Flag,Volume_Rolling_Mean,Volume_Surge,Volume_Change_3d,Drawdown_Pct,Max_Drawdown_30d,Crash_Prediction,Crash_Probability
0,2020-09-23,Amazon,149.992996,113054000,-0.041269,-0.025418,-0.103580,4.052336,9.368180,161.219348,...,-1.468553,28.277796,0,115098200.0,0.982240,-0.364337,-0.150530,-0.163259,0,0.088223
1,2020-09-24,Amazon,150.989502,110588000,0.006644,0.003676,-0.122626,3.636339,9.266949,160.164198,...,-1.797605,33.880513,0,115495600.0,0.957508,-0.096193,-0.144887,-0.163259,0,0.082590
2,2020-09-25,Amazon,154.756500,92304000,0.024949,0.047453,-0.089668,3.548090,9.039482,159.402023,...,-2.008547,40.781846,0,114538000.0,0.805881,-0.335828,-0.123553,-0.163259,0,0.069790
3,2020-09-28,Amazon,158.702499,84484000,0.025498,0.072144,-0.066950,3.990451,8.682424,158.832648,...,-2.066868,51.191209,0,113927200.0,0.741561,-0.252711,-0.101205,-0.163259,0,0.069799
4,2020-09-29,Amazon,157.244003,69916000,-0.009190,0.005078,-0.088694,3.916584,8.062282,158.067448,...,-2.045031,43.323376,0,112875800.0,0.619406,-0.367780,-0.109465,-0.163259,0,0.069782


In [20]:
df4 = df3[["Date", "Company", "Close", "Volume"] + feature_cols + ["Crash_Flag", "Crash_Prediction", "Crash_Probability"]]

df4.head(3)

,Date,Company,Close,Volume,Return_1d,Volatility_10,Volatility_20,MACD,MACD_Signal,RSI_14,...,Volume_Surge,Volume_Change_3d,Drawdown_Pct,Max_Drawdown_30d,MA_20,MA_50,MA_Crossover,Crash_Flag,Crash_Prediction,Crash_Probability
0,2020-09-23,Amazon,149.992996,113054000,-0.041269,4.052336,9.368180,-3.018738,-1.468553,28.277796,...,0.982240,-0.364337,-0.150530,-0.163259,161.219348,158.734348,2.485000,0,0,0.088223
1,2020-09-24,Amazon,150.989502,110588000,0.006644,3.636339,9.266949,-3.113813,-1.797605,33.880513,...,0.957508,-0.096193,-0.144887,-0.163259,160.164198,158.745268,1.418930,0,0,0.082590
2,2020-09-25,Amazon,154.756500,92304000,0.024949,3.548090,9.039482,-2.852315,-2.008547,40.781846,...,0.805881,-0.335828,-0.123553,-0.163259,159.402023,158.840498,0.561525,0,0,0.069790


#### **Step - 8: Exporting the Results to a CSV file**

In [21]:
df4.to_csv("Crash_Prediction_Results.csv", index = False)

files.download("Crash_Prediction_Results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

_This notebook was created and authored by Ramya Vijayalayan for educational and portfolio use only._  
© 2025 Ramya | [github.com/ramyavijayalayan-portfolio](https://github.com/ramyavijayalayan10)